In [1]:
!pip install nbformat ipython

In [ ]:
from flask import Flask, jsonify
import psycopg2
from psycopg2.extras import RealDictCursor
from config import db_name, user, password, host

app = Flask(__name__)

def get_db_connection():
    try:
        conn = psycopg2.connect(
            dbname= db_name,
            user= user ,
            password= password,
            host= host
        )
        return conn
    except psycopg2.Error as e:
        app.logger.error(f"Error connecting to database: {e}")
        return None

@app.route('/landslides/surface', methods=['GET'])
def get_landslides():
    conn = get_db_connection()
    if conn is None:
        return jsonify({"error": "Database connection error"}), 500
    
    try:
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute(" SELECT ST_AsText(geometry) AS geometry_wkt, ar_kmq , nome, ar_fr_p2, ar_fr_p1, ar_fr_p3p4, ar_frp3p4p FROM dataset ;")
        landslides = cursor.fetchall()
        cursor.close()
        conn.close()
        return jsonify(landslides)
    except psycopg2.Error as e:
        app.logger.error(f"Error executing SQL query: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=5000, use_reloader=False)
